In [ ]:
import torch
import numpy as np
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchsampler import ImbalancedDatasetSampler
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from lightning.pytorch.callbacks import ModelCheckpoint

In [ ]:
DATA_DIR = "./data/piece-images"
TORCH_MODEL_PATH = "./models/piece_classifier.pth"
ONNX_MODEL_PATH = "./models/piece_classifier.onnx"

IMAGE_SIZE = 64

In [ ]:
dataset = ImageFolder(
    DATA_DIR,
    transform=transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
    ])
)

loader = DataLoader(
    dataset,
    sampler=ImbalancedDatasetSampler(dataset),
    batch_size=1
)

mean = np.array([0.0, 0.0, 0.0])
std = np.array([0.0, 0.0, 0.0])

for image, label in loader:
    im = np.squeeze(image.numpy())
    mean += np.mean(im, axis=(1, 2))
    std += np.std(im, axis=(1, 2))

mean /= len(loader) 
std /= len(loader)

print("mean:", mean) # [0.57665089 0.5822121  0.54763596]
print("std:", std)   # [0.18085433 0.21391266 0.23309964]

In [ ]:
normalize = transforms.Normalize(
    (0.57665089, 0.5822121, 0.54763596),
    (0.18085433, 0.21391266, 0.23309964)
)
unnormalize = transforms.Compose([
    transforms.Normalize(
        (0.0, 0.0, 0.0),
        (1/0.18085433, 1/0.21391266, 1/0.23309964)
    ),
    transforms.Normalize(
        (-0.57665089, -0.5822121, -0.54763596),
        (1.0, 1.0, 1.0)
    )
])

In [ ]:
dataset = ImageFolder(
    DATA_DIR,
    transform=transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
        normalize
    ])
)
idx_to_label = list(dataset.class_to_idx.keys())
print("labels:", idx_to_label)

augmentated_dataset = ImageFolder(
    DATA_DIR,
    transform=transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.9, 1.1), ratio=(1.0, 1,0)),
        transforms.RandomAffine(degrees=(0.0, 0.0), translate=(0.1, 0.1), fill=(122, 111, 95)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
        transforms.ToTensor(),
        normalize
    ])
)

train_val_indices, test_indices = train_test_split(
    list(range(len(dataset.targets))),
    test_size=0.2,
    stratify=dataset.targets
)
train_indices, val_indices = train_test_split(
    train_val_indices,
    test_size=0.25,
    stratify=np.array(dataset.targets)[train_val_indices],
)
train_dataset = Subset(augmentated_dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

def create_loader(subset):
    return DataLoader(
        subset,
        batch_size=10,
        sampler=ImbalancedDatasetSampler(
            subset,
            labels=[subset.dataset.targets[i] for i in subset.indices]
        )
    )

train_loader = create_loader(train_dataset)
val_loader = create_loader(val_dataset)
test_loader = create_loader(test_dataset)

In [ ]:
images, targets = next(iter(train_loader))
image, target = next(zip(images, targets))
plt.title(idx_to_label[target])
plt.imshow(np.transpose(unnormalize(image).numpy(), (1, 2, 0)))
plt.axis('off')
plt.show()

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, 3,),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(64, 128, 3),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(128, 256, 3),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Linear(9216, len(idx_to_label))

    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        return x

In [ ]:
from torchsummary import summary
summary(Net(), (3, 64, 64))

In [ ]:
from typing import Any
import lightning as L
from lightning.pytorch.utilities.types import STEP_OUTPUT

class PieceClassifier(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = Net()
        self.loss_fn = nn.CrossEntropyLoss()
    
    def forward(self, inputs):
        return self.model(inputs)
    
    def training_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs)
        loss = self.loss_fn(output, target)
        metrics = {"loss": loss}
        self.log_dict(metrics, prog_bar=True, logger=True, on_epoch=True, on_step=False)
        return loss
    
    def validation_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs)
        loss = self.loss_fn(output, target)
        pred = torch.argmax(output, dim=1)
        acc = torch.sum(pred == target) * 1.0 / len(target)
        metrics = {"val_loss": loss, "val_acc": acc}
        self.log_dict(metrics, prog_bar=True, logger=True, on_epoch=True, on_step=False)
        return metrics
    
    def test_step(self, batch, batch_idx):
        inputs, target = batch
        output = self(inputs)
        loss = self.loss_fn(output, target)
        pred = torch.argmax(output, dim=1)
        acc = torch.sum(pred == target) * 1.0 / len(target)
        metrics = {"test_loss": loss, "test_acc": acc}
        self.log_dict(metrics, prog_bar=True, logger=True, on_epoch=True, on_step=False)
        return metrics

    def configure_optimizers(self):
        return torch.optim.SGD(self.model.parameters(), lr=0.1)

In [ ]:
model = PieceClassifier()
trainer = L.Trainer(
    limit_train_batches=50,
    max_epochs=120,
    callbacks=[
        ModelCheckpoint(
            monitor="val_acc",
            dirpath="./models/checkpoints",
            filename="piece-{epoch:02d}-{val_acc:.02f}",
            save_top_k=3,
            mode="max",
        )
    ]
)
trainer.fit(model, train_loader, val_loader)

In [ ]:
model = PieceClassifier.load_from_checkpoint("./models/checkpoints/piece-epoch=94-val_acc=1.00.ckpt")
trainer.test(model, test_loader)

In [ ]:
torch.save(model.model.state_dict(), TORCH_MODEL_PATH)

In [ ]:
torch_model = Net()
torch_model.load_state_dict(torch.load(TORCH_MODEL_PATH))
torch_input = next(iter(loader))[0]
torch.onnx.export(torch_model, torch_input, ONNX_MODEL_PATH)